# Cellular Automaton And Its Applications

**`Cellular Automata (CA)`** are discrete, abstract computational systems that have proved useful both as general models of complexity and as more specific representations of non-linear dynamics in a variety of scientific fields.

* Firstly, **`CA are spatially and temporally discrete`**. They are composed of a finite or denumerable set of homogenous, simple units, the atoms or cells. At each time unit, the cells instantiate one of a finite set of states. They evolve in parallel at discrete time steps, following state update functions or dynamical transition rules. The update of a cell state obtains by taking into account the states of cells in its local neighborhood. Therefore, no actions at a distance take place.


* Secondly, **`CA are abstract`**. They can be specified in purely mathematical terms and physical structures can implement them.


* Thirdly, **`CA are computational systems`**. They can compute functions and solve algorithmic problems. Despite functioning in a different way from traditional, Turing machine-like devices, CA with suitable rules can emulate a universal Turing machine and therefore compute anything computable.

<hr style="height:2px;border-width:0;color:black;background-color:black">

**In this project, we will simulate three applications of Cellular Automata namely**:

* **Snowflake Formation**
* **Forest Fire Propagation**
* **Wave Propagation**

## Snowflake Formation

**`Snowflake`** is one of the well-known examples of **`crystal formation`**. Snowflakes are collections of snow crystals, loosely bound together. Snow crystals are individual, single ice crystals, often with sixfold symmetrical shapes. These grow directly from condensing water vapor in the air, usually around a nucleus. Snowflakes can be modeled with the help of **`Cellular Automata`**. 

**`Hexagonal Cellular Automata`** imitates the growth of an ice crystal due to its hexagonal lattice structure. 


<img src = https://scx1.b-cdn.net/csz/news/800a/2008/mathmodelssn.jpg >


***
**In this section, we try to simulate the formation of Snowflake using Hexagonal Cellular Automata.** 
***

### Importing the Libraries

First we import the required modules.

In [1]:
from PIL import Image, ImageDraw
from copy import copy, deepcopy
import random
import os
import argparse
import imageio
import numpy as np
from IPython.display import Video
import shutil
from IPython.display import HTML

### Setting the Parameters

Formation of a Snowflake involves various factors such as:

#### Coefficients of Attachment Phase
* `ALPHA`: Coefficient that determines the minimum amount of ice in a cell for it to attach itself to the crystal if it has 3 crystal cells in the neighbourhood.
* `BETA`: Coefficient that determines the minimum amount of ice in a cell for it to attach itself to the crystal if it only has 1 or 2 crystal cells in the neighbourhood.
* `THETA`: Coefficient that determines the maximum amount of vapor surrounding the cell for it to still turn into a part of the crystal. 

#### Coefficients of Melting Phase
* `GAMMA`: Proportion of ice that transforms into steam.
* `MU`: Proportion of water that transforms into steam.

#### Coefficients of Freezing Phase
* `KAPPA`: The fraction used fo the evolution of the snowflake.

#### Additional Parameters
* `NUMBER`: The number of times the simulation will update
* `DIMENSION`: Dimension of final simulation.
* `RHO`: Density of steam in each cell at the begining of the simulation.
* `SIGMA`: The coefficient used for the interferences.
* `FREQUENCY`: The frequency at which the program saves the state of the snowflake.
* `APPROXIMATION`: The range which represents the distance from the initial cell where the calculations are made.
* `DEFAULT_CELL`: Original State of the cell.
    * **b**: Proportion of quasi-liquid water
    * **c**: Proportion of ice
    * **d**: Quantity of steam

In [2]:
# Coefficients of the attachment phase
ALPHA = 0.6
BETA = 0.6
THETA = 0.6

# Coefficients of the melting phase
GAMMA = 0.5 
MU = 0.5 

# Coefficients of the freezing phase
KAPPA = 0.6 

# Additional Parameters
NUMBER = 500 
DIMENSION = [150,150] 
RHO = 1.1 
SIGMA = 0.000 
FREQUENCY = 1 
APPROXIMATION = 40
DEFAULT_CELL = {"is_in_crystal":False, "b":0, "c":0, "d":RHO}

img = []

### Creating the Initial Nucleus

`create_plate` returns a 2D grid having one cell set as nucleus. 

In [3]:
def create_plate(dim=DIMENSION, initial_position=-1):
    
    plate = [[copy(DEFAULT_CELL) for j in range(dim[1])] for i in range(dim[0])]
    if initial_position == -1:
        initial_position = (dim[0]//2, dim[1]//2)
    plate[initial_position[0]][initial_position[1]] = {"is_in_crystal":True, "b":0, "c":1, "d":0, "i":0}
    return plate

### Finding the Neighbours

`generate_neighbours` returns the set of possible neighbours influencing the state change.

`get_neighbours` returns the list of correct neighbours.

In [4]:
def generate_neighbours(coordinates):
    
    x = coordinates[1]
    y = coordinates[0]
    
    if y % 2 == 0:
        return [(y, x-1), (y-1, x-1), (y-1, x), (y, x+1), (y+1, x), (y+1, x-1)]

    else:
        return [(y, x-1), (y-1, x), (y-1, x+1), (y, x+1), (y+1, x+1), (y+1, x)]

    

def get_neighbours(coordinates, dim=DIMENSION):
    
    list_neighbours = [x for x in generate_neighbours(coordinates) if all(i >= 0 for i in x) and all(i < dim[j] for i in x for j in range(2))]
    return list_neighbours

NEIGHBOURS = {}
for i in range(DIMENSION[0]):
    for j in range(DIMENSION[1]):
        NEIGHBOURS[(i,j)] = get_neighbours((i,j))

### Diffusion Process

`diffusion_cell` returns the state of a cell after diffusion.

`diffusion` returns the state of entire plate after diffusion.

In [5]:
def diffusion_cell(y, x, cell, changes_to_make, plate_in):
    
    if cell["is_in_crystal"] == False:
        neighbours = NEIGHBOURS[(y, x)]
        steam = cell["d"]
        for (y2,x2) in neighbours:
            dic = plate_in[y2][x2]
            if dic["is_in_crystal"] == True:
                steam += cell["d"]
            else:
                steam += dic["d"] 
        changes_to_make[(y, x)] = steam / (1+len(neighbours))
        return changes_to_make

    

def diffusion(plate_in, init_pos, max_point, approximation=0):
    
    changes_to_make = {}
    if not approximation:
        for (y,x) in NEIGHBOURS:
            cell = plate_in[y][x]
            diffusion_cell(y, x, cell, changes_to_make, plate_in)
    else:
        for y in range(max(0, init_pos[0] - approximation - max_point), min(DIMENSION[0], init_pos[0]+approximation + max_point)):
            for x in range(max(0, init_pos[1] - approximation - max_point), min(DIMENSION[1], init_pos[1] + approximation + max_point)):
                cell = plate_in[y][x]
                diffusion_cell(y, x, cell, changes_to_make, plate_in)
    for coord, value in changes_to_make.items(): 
        plate_in[coord[0]][coord[1]]["d"] = value
    return plate_in

### Phases

`freezing` returns the state of a cell after freezing phase.

`attachment` returns the state of a cell after attachment phase.

`melting` returns the state of a cell after melting phase.

`interference` returns the state of a cell after interference phase.

In [6]:
def attachment(di, cell_at_border, neighbours, ind, alpha=ALPHA, beta=BETA, theta=THETA):
    
    x, y = cell_at_border[1], cell_at_border[0]
    
    cristal_neighbours = 0
    test_with_theta = 0
    for neigh_coord, neigh_di in neighbours.items():
        if neigh_di["is_in_crystal"] == True:
            cristal_neighbours += 1     
        test_with_theta += neigh_di["d"]
            
    if (((cristal_neighbours in (1, 2)) and (di["b"] > beta))
        or ((cristal_neighbours == 3) and ((di["b"] >= 1) 
        or ((test_with_theta < theta) and (di["b"] >= alpha))))
        or cristal_neighbours > 3): # We attach the cell to the crystal
        di_out = deepcopy(di)
        di_out["c"] = di["c"] + di["b"]
        di_out["b"] = 0
        di_out["d"] = 0
        di_out["is_in_crystal"] = True
        di_out["i"] = ind
        return di_out
    else:
        return None
    
    
    
def freezing(di, k=KAPPA):
    
    di["b"] = di["b"] + (1 - k) * di["d"]
    di["c"] = di["c"] + k * di["d"]
    di["d"] = 0
    return di

def melting(di, mu=MU, gamma=GAMMA):
   
    di["d"] = di["d"] + mu * di["b"] + gamma * di["c"]
    di["b"] = (1-mu) * di["b"]
    di["c"] = (1-gamma) * di["c"]
    return di



def interference(plate, sigma=SIGMA):
    
    for (y,x) in NEIGHBOURS:
        cell = plate[y][x]
        if cell["is_in_crystal"] == False:
            cell["d"] = cell["d"] * (1 + (random.random()- 0.5) * sigma)
    return None

### Additional Functions

`savestates` is used to save the images of snowflake formation at different stages.

`create_video` is used to generate simulation video for snowflake formation.

In [7]:
def savestates(plate, filename, n, newpath, number=NUMBER):
    
    index_number = str(n).zfill(len(str(number))) # Adds leading zeros in front of the index (instead of 50 we would get 050)
    
    # Creating the Hexagon Image
    x = 12*DIMENSION[1]+6
    y = DIMENSION[0]*11+3
    snowflake = Image.new("RGB", (x, y), color=0)
    for y in range(DIMENSION[0]):
        for x in range(DIMENSION[1]):
            
            
            x_ = 0 if (y % 2 == 0) else 6
            
            shape = [
                (12*x +6  +x_, y*10    ),
                (12*x +12 +x_, y*10 +3 ),
                (12*x +12 +x_, y*10 +11),
                (12*x +6  +x_, y*10 +14),
                (12*x     +x_, y*10 +11),
                (12*x     +x_, y*10 +3 )
            ]
            
            d = plate[y][x]
            if d["is_in_crystal"] == False:
                ImageDraw.Draw(snowflake).polygon(xy=shape, fill=(0,0,255 - int((d["d"] / RHO)*255)), outline=(0,0,255 - int((d["d"] / RHO)*255)), )
            else:
                ImageDraw.Draw(snowflake).polygon(xy=shape, fill=(255 - (int(d["i"]/(2*NUMBER)*255)),255 - (int(d["i"]/(5*NUMBER)*255)), 255), outline=(255 - (int(d["i"]/(2*NUMBER)*255)),255 - (int(d["i"]/(5*NUMBER)*255)), 255))
    snowflake.save(newpath + "/Hexagons/" + filename + index_number + ".jpeg", format="JPEG")
    img.append(np.array(snowflake))
    return



def create_video(path):
    
    newpath = path + "/Hexagons/" 
    list_pictures = [f for f in os.listdir(newpath) if (os.path.isfile(os.path.join(newpath, f)) and ("jpeg" in f or "png" in f))]
    list_pictures.sort()
    
    with imageio.get_writer("Snowflake Simulation.mp4", fps = 30, quality=10) as writer:
        for filename in list_pictures:
            image = Image.open(newpath + filename)
            image = image.rotate(30)
            writer.append_data(np.array(image))

### Snowflake Generator Function

`snowflake_creator` uses all the above mentioned functions to simulate formation of snowflake.

In [8]:
def snowflake_creator(number=NUMBER, dim=DIMENSION, init_pos=-1,
                    alpha=ALPHA, beta=BETA, theta=THETA,
                    mu=MU, gamma=GAMMA, kappa=KAPPA,
                    sigma=SIGMA, frequency=FREQUENCY):
    
    print("Creating plate...")
    plate = create_plate(initial_position = init_pos)
    print("Plate successfully created !")
    
    newpath = "./snowflake_states"
    
    # Creates directories if they do not exist    
    try:
        shutil.rmtree('snowflake_states')
    except:
        print("File doesn't exists")
        
    os.makedirs(newpath)
        
    if not os.path.exists(newpath + "/Hexagons"):
        os.makedirs(newpath + "/Hexagons")


    if init_pos == -1: 
        init_pos = (dim[0]//2, dim[1]//2)
    cells_at_border = set(NEIGHBOURS[(init_pos[0], init_pos[1])]) 
    max_point = 0
    len_total = len(str(number))
    
    # Runs the simulation `number` times
    print("Running simulation...")
    print("\n    Frames    |   Distance")
    print("- - - - - - - - - - - - - - -")
    for i in range(number):
        #DIFFUSION
        plate = diffusion(plate, init_pos, max_point, approximation=APPROXIMATION)

        changes_to_make = {}
        for cell in cells_at_border: 
            cell_di = plate[cell[0]][cell[1]] 
            
            # FREEZING
            cell_di = freezing(cell_di)
            
            # ATTACHMENT
            neighbours = {} 
            for y, x in NEIGHBOURS[(cell[0], cell[1])]:
                neighbours[(y,x)] = plate[y][x]
            new_cell = attachment(cell_di, cell, neighbours, i)
            if new_cell: 
                changes_to_make[(cell[0], cell[1])] = new_cell
            
            # MELTING
            cell_di = melting(cell_di)
            
        # INTERFERENCE
        if sigma:
            interference(plate)
        
        for coord, di in changes_to_make.items(): 
            plate[coord[0]][coord[1]] = di 
            max_point = max(abs(init_pos[0] - coord[0]), abs(init_pos[1] - coord[1]), max_point)
            neighbours = {}
            for y, x in NEIGHBOURS[(coord[0], coord[1])]: 
                neighbours[(y,x)] = plate[y][x] 
            
            for neigh_coord, di in neighbours.items():
                if (di["is_in_crystal"] == False 
                    and (not neigh_coord in changes_to_make)):
                    cells_at_border.add(neigh_coord) 
            cells_at_border.remove(coord) 
        
        
        # Saves the state of the plate
        if i % frequency == 0:
            savestates(plate, "snowflake", i, newpath)
            print("{frames:{longueur}d} / {total} |   {distance}".format(longueur = len_total + 2, frames=i, total=number, distance=max_point))
    savestates(plate, "snowflake", i, newpath)
    print("Simulation done !")
    print("Generating simulation video...")
    create_video(newpath) # Creates a gif from all the pictures saved from the plate
    print("Video successfully generated !")
    return

In [9]:
if __name__ == '__main__':
    snowflake_creator() # Runs the simulation

Creating plate...
Plate successfully created !
Running simulation...

    Frames    |   Distance
- - - - - - - - - - - - - - -
    0 / 500 |   0
    1 / 500 |   0
    2 / 500 |   0
    3 / 500 |   1
    4 / 500 |   1
    5 / 500 |   1
    6 / 500 |   1
    7 / 500 |   1
    8 / 500 |   1
    9 / 500 |   2
   10 / 500 |   2
   11 / 500 |   2
   12 / 500 |   2
   13 / 500 |   2
   14 / 500 |   2
   15 / 500 |   2
   16 / 500 |   2
   17 / 500 |   3
   18 / 500 |   3
   19 / 500 |   3
   20 / 500 |   3
   21 / 500 |   3
   22 / 500 |   3
   23 / 500 |   3
   24 / 500 |   4
   25 / 500 |   4
   26 / 500 |   4
   27 / 500 |   4
   28 / 500 |   4
   29 / 500 |   4
   30 / 500 |   4
   31 / 500 |   4
   32 / 500 |   4
   33 / 500 |   5
   34 / 500 |   5
   35 / 500 |   5
   36 / 500 |   5
   37 / 500 |   5
   38 / 500 |   5
   39 / 500 |   5
   40 / 500 |   5
   41 / 500 |   6
   42 / 500 |   6
   43 / 500 |   6
   44 / 500 |   6
   45 / 500 |   6
   46 / 500 |   6
   47 / 500 |   6
   48 / 5

  429 / 500 |   47
  430 / 500 |   47
  431 / 500 |   47
  432 / 500 |   47
  433 / 500 |   47
  434 / 500 |   48
  435 / 500 |   48
  436 / 500 |   48
  437 / 500 |   48
  438 / 500 |   48
  439 / 500 |   48
  440 / 500 |   48
  441 / 500 |   48
  442 / 500 |   48
  443 / 500 |   49
  444 / 500 |   49
  445 / 500 |   49
  446 / 500 |   49
  447 / 500 |   49
  448 / 500 |   49
  449 / 500 |   49
  450 / 500 |   49
  451 / 500 |   50
  452 / 500 |   50
  453 / 500 |   50
  454 / 500 |   50
  455 / 500 |   50
  456 / 500 |   50
  457 / 500 |   50
  458 / 500 |   50
  459 / 500 |   50
  460 / 500 |   50
  461 / 500 |   50
  462 / 500 |   51
  463 / 500 |   51
  464 / 500 |   51
  465 / 500 |   51
  466 / 500 |   51
  467 / 500 |   51
  468 / 500 |   51
  469 / 500 |   51
  470 / 500 |   52
  471 / 500 |   52
  472 / 500 |   52
  473 / 500 |   52
  474 / 500 |   52
  475 / 500 |   52
  476 / 500 |   52
  477 / 500 |   52
  478 / 500 |   53
  479 / 500 |   53
  480 / 500 |   53
  481 / 500 

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1806, 1653) to (1808, 1664) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Video successfully generated !


### Final Simulation

In [10]:
HTML("""
    <video height = "500" width = "500" controls>
        <source src="Snowflake Simulation.mp4" type="video/mp4">
    </video>
""")

## Forest Fire

The most common hazard in forests is forests fire. **`Forests Fires`** are as old as the forests themselves. They pose a threat not only to the forest wealth but also to the entire regime to **`fauna`** and **`flora`** seriously disturbing the bio-diversity and the ecology and environment of a region. During summer, when there is no rain for months, the forests become littered with dry senescent leaves and twinges, which could burst into flames ignited by the slightest spark.

**`Cellular Automata`** have been successfully applied to **`simulate the propagation of wildfires`** with the aim of **`assisting fire managers`** in defining fire suppression tactics and in planning fire risk management policies. Cellular Automata whose rules for updation is driven by some external probabilities are called  **`Probabilistic Cellular Automata`**.


<img src = 'https://d2ouvy59p0dg6k.cloudfront.net/img/original/chiangmai_wildfire__27_mar_2020.jpg'>


***
**In this section, we try to simulate the propagation of wildfires using Cellular Automata.** 
***

### Importing the Libraries

In [11]:
import numpy as np
import scipy as sp
import scipy.sparse
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib
from PIL import Image
import os
import shutil
import imageio

### Setting the Parameters

In [12]:
path = os.getcwd()
empty = 0
unburned = 1
burning = 2 
burned = 3
cvals  = [0, 1, 2, 3]
colors = ['darkgreen', 'limegreen', '#FFD20F', '#EA482B']
norm = plt.Normalize(min(cvals),max(cvals))
tuples = list(zip(map(norm,cvals), colors))
cmap1 = matplotlib.colors.LinearSegmentedColormap.from_list("", tuples)

### Creating Forest

In [13]:
def create_world(n, q):
    world = np.zeros((n+2, n+2))
    forest = world[1:-1, 1:-1]
    forest[:, :] = np.random.choice([0, 1], p=[1-q, q], size=(n, n))
    return world

def show_world(W, title=None, **args):
    if 'cmap' not in args:
        args['cmap'] = cmap1
    if 'vmin' not in args and 'vmax' not in args:
        args['vmin'] = 0
        args['vmax'] = 3
    plt.figure(figsize=(8, 8))
    plt.matshow(W, fignum=1, **args)
    plt.xlabel('column')
    plt.ylabel('row')
    plt.colorbar()
    if title is None:
        num_trees = (W > 0).sum()
        if num_trees > 0:
            num_burnt = (W == 3).sum()
            percent = num_burnt / num_trees * 1e2
            title = '{} / {} ~= {:.1f}%'.format(num_burnt, num_trees, percent)
        else:
            title = ''
    plt.title(title)
    pass

In [14]:
def is_empty(W):
    return W == empty

# This tells us if something was vegetation at any point, burned or unburned!
def is_vegetation(W):
    return W > empty

def is_unburned(W):
    return W == 1

def is_burning(W):
    return W == 2

def is_burned(W):
    return W == 3

def count(W, cond_fun):
    return cond_fun(W).sum()

def summarize_world(W):
    def suffix(n):
        return (1, "tree") if n == 1 else (n, "trees")
    m, n = W.shape[0]-2, W.shape[1]-2
    n_trees = count(W, is_vegetation)
    n_unburned = count(W, is_unburned)
    n_burning = count(W, is_burning)
    n_burned = count(W, is_burned)
    
    print("The world has dimensions: {} x {}".format(m, n))
    print("There are {} cell(s) that have had vegetation in them".format(n_trees))
    print("There are {} cell(s) of vegetation that are unburned".format(n_unburned))
    print("There are {} cell(s) of vegetation on fire".format(n_burning))
    print("There are {} cell(s) of vegetation completely burned".format(n_burned))

### Starting Fire

In [15]:
def start_fire(W, cells=None):
    
    W_new = W.copy()
    
    if cells == None:
        F = W[1:-1, 1:-1]
        W_new = W.copy()
        F_new = W_new[1:-1, 1:-1]
        I, J = np.where(is_unburned(F)) # Positions of all trees
        if len(I) > 0:
            k = np.random.choice(range(len(I))) # Index of tree to ignite
            i, j = I[k], J[k]
            assert F_new[i, j] == 1, "Attempting to ignite a non-tree?"
            F_new[i, j] += 1
    else:
        W_new = W.copy()
        
        for x,y in cells:
            W_new[x,y] = 2
    
    return W_new

### Spreading Fire

In [16]:
def spread_fire(W):
    W_new = W.copy()
    Vegetation = is_unburned(W)
    Fires = is_burning(W)
    W_new[1:-1, 1:-1] += Vegetation[1:-1, 1:-1] \
                         & (
                              Fires[:-2, :-2]  | Fires[1:-1, :-2] | Fires[2:, :-2]
                            | Fires[:-2, 1:-1] |                    Fires[2:, 1:-1]
                            | Fires[:-2, 2:]   | Fires[1:-1, 2:]  | Fires[2:, 2:]
                           )

    W_new[1:-1, 1:-1] += Fires[1:-1, 1:-1]
    return W_new

In [17]:
def spread_fire_p(W, fire_spread_prob=0.4):
    W_new = W.copy()
    Vegetation = is_unburned(W)
    Fires = is_burning(W)
    num_neighbors_on_fire = (Fires[:-2, :-2].astype(int) + Fires[1:-1, :-2] + Fires[2:, :-2] 
                             + Fires[:-2, 1:-1]                             + Fires[2:, 1:-1] 
                             + Fires[:-2, 2:]            + Fires[1:-1, 2:]  + Fires[2:, 2:])

    num_neighbors_on_fire = np.multiply(num_neighbors_on_fire, Vegetation[1:-1, 1:-1].astype(int))
    fire_prob_matrix = np.ones(num_neighbors_on_fire.shape) - fire_spread_prob
    fire_prob_matrix = 1.0 - np.power(fire_prob_matrix, num_neighbors_on_fire)
    fire_prob_matrix = np.multiply(fire_prob_matrix, Vegetation[1:-1, 1:-1].astype(int))
    randys = np.random.rand(*fire_prob_matrix.shape)
    new_on_fires = randys < fire_prob_matrix
    W_new[1:-1, 1:-1] += new_on_fires
    W_new[1:-1, 1:-1] += Fires[1:-1, 1:-1]
    return W_new

### Forest Fire Simulator Function

In [18]:
def simulate(W0, t_max=None, inplace=False, spread_prob=None):
    if t_max is None:
        n_max = max(W0.shape)
        t_max = n_max * ((2*n_max-1) // 2)
    W = np.zeros((W0.shape[0], W0.shape[1], 2 if inplace else t_max+1))
    t_cur = 0
    W[:, :, t_cur] = W0
    for t in range(t_max):
        t_next = (t_cur+1)%2 if inplace else t+1
        
        if spread_prob:
            W[:, :, t_next] = spread_fire_p(W[:, :, t_cur], fire_spread_prob=spread_prob)
        else:
            W[:, :, t_next] = spread_fire(W[:, :, t_cur])
            
        if (W[:, :, t_cur] == W[:, :, t_next]).all():
            t_cur = t_next
            break
        t_cur = t_next
    return (W[:, :, t_cur], t) if inplace else W[:, :, :t_cur+1]
    
    
def run_simulation(n, q, **args):
    return simulate(start_fire(create_world(n, q)), **args)

In [19]:
W = run_simulation(30, 0.7, spread_prob = 0.6)

### Creating Simulation Video 

In [20]:
images = []
for k in range(W.shape[2]):
    img=[]
    w = W[:,:,k]
    
    for i in range(w.shape[0]):
        for j in range(w.shape[1]):
            if w[i][j]==0:
                img.append((0,100,0))
            elif w[i][j]==1:
                img.append((50,205,50))
            elif w[i][j]==2:
                img.append((255,210,15))
            else:
                img.append((255,140,0))
    forest = Image.new('RGB',[w.shape[0],w.shape[1]],color=0)
    forest.putdata(img)
    images.append(np.array(forest))

In [21]:
try:
    shutil.rmtree(path+'/Forest_Fire_States')
except:
    print("File doesn't exists")
    
os.mkdir(path+'/Forest_Fire_States')

In [22]:
%%capture
for i in range(len(images)):
    index = str(i).zfill(len(str(len(images))))
    plt.figure(figsize=(10,10))
    plt.imshow(images[i])
    plt.savefig(path+'/Forest_Fire_States/img'+index+'.jpeg')

In [23]:
path = os.getcwd()
newpath = path + "/Forest_Fire_States/" 
list_pictures = [f for f in os.listdir(newpath) if (os.path.isfile(os.path.join(newpath, f)) and ("jpeg" in f or "png" in f))]
list_pictures.sort()

with imageio.get_writer("Forest Fire Simulation.mp4", fps = 2, quality=10) as writer:
    for filename in list_pictures:
        image = imageio.imread(newpath + filename)
        writer.append_data(image)

### Final Simulation

In [24]:
from IPython.display import Video 
Video("Forest Fire Simulation.mp4",height=500,width=500)

## Wave Propagation Simulation

A **`wave`** is a propagating dynamic disturbance (change from equilibrium) of one or more quantities, sometimes as described by a wave equation. In physical waves, at least two field quantities in the wave medium are involved. Waves can be periodic, in which case those quantities oscillate repeatedly about an equilibrium (resting) value at some frequency. **`Cellular Automata`** can be used to **`simulate the Propagation of Waves`**. Cellular Automata is used to update the Cellular Velocities at each time step.

<img src = "https://previews.123rf.com/images/rustyphil/rustyphil0711/rustyphil071100033/2042162-a-top-down-view-of-the-rings-of-a-perfect-water-like-ripple-in-silver-or-chrome.jpg" height = 400 width = 400>


***
**In this section, we try to simulate the Wave Propagation using Cellular Automata.** 
***

### Importing the Libraries

In [25]:
import sys
from numpy.core.umath import pi
from numpy.ma import sin
from sys import argv
from tqdm import tqdm
import os
import matplotlib
from matplotlib.animation import FuncAnimation, FFMpegWriter
from matplotlib.colors import LinearSegmentedColormap, colorConverter
import matplotlib.pyplot as plt
import shutil
import imageio
import numpy as np

### Setting the Parameters

In [26]:
scale = 25  
size_x = 6 * scale
size_y = 4 * scale
damping = 0.99
omega = 3 / (2 * pi)
initial_P = 200
vertPos = size_y - 3 * scale
horizPos = 2 * scale
wallTop = size_y - 3 * scale
wall_x_pos = 3 * scale
max_pressure = initial_P / 2
min_presure = -initial_P / 2

### Building Simulation

In [27]:
class Simulation:
    def __init__(self):
        self.frame = 0
        self.pressure = [[0.0 for x in range(size_x)] for y in range(size_y)]
        # outflow velocities from each cell
        self._velocities = [[[0.0, 0.0, 0.0, 0.0] for x in range(size_x)] for y in range(size_y)]
        self.pressure[vertPos][horizPos] = initial_P

    def updateV(self):
        """Recalculate outflow velocities from every cell based on pressure difference with each neighbour"""
        V = self._velocities
        P = self.pressure
        for i in range(size_y):
            for j in range(size_x):
                if wall[i][j] == 1:
                    V[i][j][0] = V[i][j][1] = V[i][j][2] = V[i][j][3] = 0.0
                    continue
                cell_pressure = P[i][j]
                V[i][j][0] = V[i][j][0] + cell_pressure - P[i - 1][j] if i > 0 else cell_pressure
                V[i][j][1] = V[i][j][1] + cell_pressure - P[i][j + 1] if j < size_x - 1 else cell_pressure
                V[i][j][2] = V[i][j][2] + cell_pressure - P[i + 1][j] if i < size_y - 1 else cell_pressure
                V[i][j][3] = V[i][j][3] + cell_pressure - P[i][j - 1] if j > 0 else cell_pressure

    def updateP(self):
        for i in range(size_y):
            for j in range(size_x):
                self.pressure[i][j] -= 0.5 * damping * sum(self._velocities[i][j])

    def step(self):
        self.pressure[vertPos][horizPos] = initial_P * sin(omega * self.frame)
        self.updateV()
        self.updateP()
        self.frame += 1

### Creating the Walls

In [28]:
def create_walls(no_of_walls):
    if no_of_walls == 1:
        wall = [[1 if x == wall_x_pos and wallTop < y < size_y else 0
                 for x in range(size_x)] for y in range(size_y)]
    elif no_of_walls == 2:
        wall = [[1 if (x == wall_x_pos and wallTop + scale < y < size_y) or
                      (wall_x_pos - scale < x < wall_x_pos and
                       x - wall_x_pos == y - wallTop - scale - 1) or
                      (wall_x_pos < x < wall_x_pos + scale and
                       x - wall_x_pos == -y + wallTop + scale + 1)
                 else 0
                 for x in range(size_x)] for y in range(size_y)]
    elif no_of_walls == 3:
        wall = [[1 if (x <= 1/3*size_x and y == 3/5*size_y) or 
                      (x == 3/5*size_x and y <= 1/3*size_y) or
                      (x == 3/5*size_x and y >= 2/3*size_y)
                 else 0
                 for x in range(size_x)] for y in range(size_y)]
    else:
        wall = [[1 if (x == wall_x_pos and wallTop + scale < y < size_y) or
                      (wall_x_pos - scale < x < wall_x_pos and
                       x - wall_x_pos == y - wallTop - scale - 1) or
                      (wall_x_pos < x < wall_x_pos + scale and
                       x - wall_x_pos == -y + wallTop + scale + 1) or
                      (wall_x_pos - 0.75 * scale < x < wall_x_pos - scale / 2 and
                       x - wall_x_pos == -y + wallTop - scale / 2 + 1) or
                      (wall_x_pos + scale / 2 < x < wall_x_pos + 0.75 * scale and
                       x - wall_x_pos == y - wallTop + scale / 2 - 1)
                 else 0
                 for x in range(size_x)] for y in range(size_y)]
    
    return wall

In [29]:
def create_walls1(no_of_walls):
    if no_of_walls == 1:
        wall = [[1 if x == wall_x_pos and wallTop < y < size_y else 0
                 for x in range(size_x)] for y in range(size_y)]
    elif no_of_walls == 2:
        wall = [[1 if (x == wall_x_pos and wallTop + scale < y < size_y) or
                      (wall_x_pos - scale < x < wall_x_pos and
                       x - wall_x_pos == y - wallTop - scale - 1) or
                      (wall_x_pos < x < wall_x_pos + scale and
                       x - wall_x_pos == -y + wallTop + scale + 1)
                 else 0
                 for x in range(size_x)] for y in range(size_y)]
    elif no_of_walls == 3:
        wall = [[1 if (((x - 40)**2 + (y - 50)**2) >= 0 and ((x - 40)**2 + (y - 50)**2) <= 4)
                 or (((x - 100)**2 + (y - 30)**2) >= 0 and ((x - 100)**2 + (y - 30)**2) <= 4)
                 else 0
                 for x in range(size_x)] for y in range(size_y)]
    else:
        wall = [[1 if (x == wall_x_pos and wallTop + scale < y < size_y) or
                      (wall_x_pos - scale < x < wall_x_pos and
                       x - wall_x_pos == y - wallTop - scale - 1) or
                      (wall_x_pos < x < wall_x_pos + scale and
                       x - wall_x_pos == -y + wallTop + scale + 1) or
                      (wall_x_pos - 0.75 * scale < x < wall_x_pos - scale / 2 and
                       x - wall_x_pos == -y + wallTop - scale / 2 + 1) or
                      (wall_x_pos + scale / 2 < x < wall_x_pos + 0.75 * scale and
                       x - wall_x_pos == y - wallTop + scale / 2 - 1)
                 else 0
                 for x in range(size_x)] for y in range(size_y)]
    
    return wall

### Running the Simulation

In [30]:
%%capture
no_of_wall = 3
wall = create_walls1(no_of_wall)
# wall = np.zeros((size_y, size_x))
path = os.getcwd()
matplotlib.use('Qt5Agg')

simulation = Simulation()
figure = plt.figure(figsize = (8, 8))
figure.patch.set_facecolor('#E0E0E0')
cvals  = [-3., -2., -1., 0, 1, 2, 3]
colors = ['yellow', 'orange', '#FD0E35', 'dodgerblue', 'navy', 'lawngreen', 'green']
norm = plt.Normalize(min(cvals),max(cvals))
tuples = list(zip(map(norm,cvals), colors))
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", tuples)
ca_plot = plt.imshow(simulation.pressure, cmap = cmap, interpolation = 'bilinear', vmin=min_presure, vmax=max_pressure)
# plt.colorbar(ca_plot)
transparent = colorConverter.to_rgba('black', alpha=0)
wall_colormap = LinearSegmentedColormap.from_list('my_colormap', [transparent, 'black'], 2)
plt.imshow(wall, cmap=wall_colormap, zorder=2)

try:
    shutil.rmtree(path + '\Wave_Propagation_Images')
except:
    print("File doesn't exists")
    
os.mkdir(path + '\Wave_Propagation_Images')

for i in range(200):
    index = str(i).zfill(len(str(200)))
    simulation.step()
    plt.imshow(simulation.pressure, cmap = cmap, interpolation = 'bilinear', vmin=min_presure, vmax=max_pressure)
    plt.savefig(path+'\Wave_Propagation_Images\img'+index+'.jpeg')

In [31]:
path = os.getcwd()
newpath = path + "/Wave_Propagation_Images/"
list_pictures = [f for f in os.listdir(newpath) if (os.path.isfile(os.path.join(newpath, f)) and ("jpeg" in f or "png" in f))]
list_pictures.sort()

with imageio.get_writer("Wave Propagation Simulations1.mp4", fps = 8, quality=10) as writer:
    for filename in list_pictures:
        image = imageio.imread(newpath + filename)
        writer.append_data(image)

In [32]:
%%capture
# no_of_wall = 3
# wall = create_walls1(no_of_wall)
wall = np.zeros((size_y, size_x))
path = os.getcwd()
matplotlib.use('Qt5Agg')

simulation = Simulation()
figure = plt.figure(figsize = (8, 8))
figure.patch.set_facecolor('#E0E0E0')
cvals  = [-3., -2., -1., 0, 1, 2, 3]
colors = ['yellow', 'orange', '#FD0E35', 'dodgerblue', 'navy', 'lawngreen', 'green']
norm = plt.Normalize(min(cvals),max(cvals))
tuples = list(zip(map(norm,cvals), colors))
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", tuples)
ca_plot = plt.imshow(simulation.pressure, cmap = cmap, interpolation = 'bilinear', vmin=min_presure, vmax=max_pressure)
transparent = colorConverter.to_rgba('black', alpha=0)
wall_colormap = LinearSegmentedColormap.from_list('my_colormap', [transparent, 'black'], 2)
plt.imshow(wall, cmap=wall_colormap, zorder=2)

try:
    shutil.rmtree(path + '\Wave_Propagation_Images without Damage')
except:
    print("File doesn't exists")
    
os.mkdir(path + '\Wave_Propagation_Images without Damage')

for i in range(200):
    index = str(i).zfill(len(str(200)))
    simulation.step()
    plt.imshow(simulation.pressure, cmap = cmap, interpolation = 'bilinear', vmin=min_presure, vmax=max_pressure)
    plt.savefig(path+'\Wave_Propagation_Images without Damage\img'+index+'.jpeg')

In [33]:
path = os.getcwd()
newpath = path + "/Wave_Propagation_Images without Damage/"
list_pictures = [f for f in os.listdir(newpath) if (os.path.isfile(os.path.join(newpath, f)) and ("jpeg" in f or "png" in f))]
list_pictures.sort()

with imageio.get_writer("Wave Propagation Simulations without Damage.mp4", fps = 8, quality=10) as writer:
    for filename in list_pictures:
        image = imageio.imread(newpath + filename)
        writer.append_data(image)

In [34]:
%%capture
no_of_wall = 3
wall = create_walls(no_of_wall)
# wall = np.zeros((size_y, size_x))
path = os.getcwd()
matplotlib.use('Qt5Agg')

simulation = Simulation()
figure = plt.figure(figsize = (8, 8))
figure.patch.set_facecolor('#E0E0E0')
cvals  = [-3., -2., -1., 0, 1, 2, 3]
colors = ['yellow', 'orange', '#FD0E35', 'dodgerblue', 'navy', 'lawngreen', 'green']
norm = plt.Normalize(min(cvals),max(cvals))
tuples = list(zip(map(norm,cvals), colors))
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", tuples)
ca_plot = plt.imshow(simulation.pressure, cmap = cmap, interpolation = 'bilinear', vmin=min_presure, vmax=max_pressure)
transparent = colorConverter.to_rgba('black', alpha=0)
wall_colormap = LinearSegmentedColormap.from_list('my_colormap', [transparent, 'black'], 2)
plt.imshow(wall, cmap=wall_colormap, zorder=2)

try:
    shutil.rmtree(path + '\Wave_Propagation_Images Barrier')
except:
    print("File doesn't exists")
    
os.mkdir(path + '\Wave_Propagation_Images Barrier')

for i in range(200):
    index = str(i).zfill(len(str(200)))
    simulation.step()
    plt.imshow(simulation.pressure, cmap = cmap, interpolation = 'bilinear', vmin=min_presure, vmax=max_pressure)
    plt.savefig(path+'\Wave_Propagation_Images Barrier\img'+index+'.jpeg')

In [35]:
path = os.getcwd()
newpath = path + "/Wave_Propagation_Images Barrier/"
list_pictures = [f for f in os.listdir(newpath) if (os.path.isfile(os.path.join(newpath, f)) and ("jpeg" in f or "png" in f))]
list_pictures.sort()

with imageio.get_writer("Wave Propagation Simulations Barrier.mp4", fps = 8, quality=10) as writer:
    for filename in list_pictures:
        image = imageio.imread(newpath + filename)
        writer.append_data(image)

### Final Simulation

In [36]:
%%capture
from IPython.display import Video 
Video("Wave Propagation Simulations without Damage.mp4", height = 500, width = 500)

In [37]:
%%capture
Video("Wave Propagation Simulations1.mp4", height = 500, width = 500)

In [38]:
from PIL import Image

def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

list_pictures = []

try:
    shutil.rmtree(path + '\Wave_Propagation_Images Final')
except:
    print("File doesn't exists")
    
os.mkdir(path + '\Wave_Propagation_Images Final')

for i in range(200):
    index = str(i).zfill(len(str(200)))
    im1 = Image.open(path+'\Wave_Propagation_Images\img'+index+'.jpeg')
    im2 = Image.open(path+'\Wave_Propagation_Images without Damage\img'+index+'.jpeg')
    im = get_concat_h(im1, im2)
    im.save(path+'\Wave_Propagation_Images Final\img'+index+'.jpeg', quality = 100)
    
path = os.getcwd()
newpath = path + "/Wave_Propagation_Images Final/"
list_pictures = [f for f in os.listdir(newpath) if (os.path.isfile(os.path.join(newpath, f)) and ("jpeg" in f or "png" in f))]
list_pictures.sort()

with imageio.get_writer("Wave Propagation Simulations Final.mp4", fps = 8, quality=10) as writer:
    for filename in list_pictures:
        image = imageio.imread(newpath + filename)
        writer.append_data(image)

In [39]:
Video("Wave Propagation Simulations Barrier.mp4", height = 500, width = 500)

In [40]:
from IPython.display import Video 
Video("Wave Propagation Simulations Final.mp4", height = 500, width = 500)